In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graduate-admissions/Admission_Predict.csv
/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv


In [2]:
df=pd.read_csv('/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv')

In [3]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          500 non-null    int64  
 2   TOEFL Score        500 non-null    int64  
 3   University Rating  500 non-null    int64  
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 35.3 KB


In [5]:
df.duplicated().sum()

0

In [6]:
df.drop(columns=['Serial No.'],inplace=True)

In [7]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [21]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [19]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1
496,337,117,5,5.0,5.0,9.87,1
497,330,120,5,4.5,5.0,9.56,1
498,312,103,4,4.0,5.0,8.43,0


In [22]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit , Length: 500, dtype: float64

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [25]:
X_train_scaled

array([[0.4       , 0.42857143, 0.5       , ..., 0.57142857, 0.50320513,
        0.        ],
       [0.56      , 0.64285714, 0.        , ..., 0.57142857, 0.55769231,
        1.        ],
       [0.2       , 0.32142857, 0.5       , ..., 0.28571429, 0.34615385,
        0.        ],
       ...,
       [0.7       , 0.53571429, 0.5       , ..., 0.57142857, 0.74038462,
        1.        ],
       [0.72      , 0.67857143, 1.        , ..., 0.71428571, 0.77884615,
        1.        ],
       [0.2       , 0.46428571, 0.        , ..., 0.14285714, 0.32051282,
        0.        ]])

In [27]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [28]:
model=Sequential()
model.add(Dense(7,activation='relu',input_dim=7))
model.add(Dense(1,activation='linear'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 56        
                                                                 
 dense_1 (Dense)             (None, 1)                 8         
                                                                 
Total params: 64 (256.00 Byte)
Trainable params: 64 (256.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model.compile(loss='mean_squared_error',optimizer='Adam')

In [36]:
history=model.fit(X_train_scaled,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 1s 24ms/step - loss: 0.6041 - val_loss: 0.4942
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3802 - val_loss: 0.2991
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2247 - val_loss: 0.1676
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 0.1238 - val_loss: 0.0867
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0644 - val_loss: 0.0427
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0331 - val_loss: 0.0225
Epoch 7/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0198 - val_loss: 0.0146
Epoch 8/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0145 - val_loss: 0.0123
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.0131 - val_loss: 0.0119
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 0.0127 - val_loss: 0.0119


In [39]:
y_pred=model.predict(X_test_scaled)

4/4 [==============================] - 0s 2ms/step


In [40]:
y_pred

array([[0.5277291 ],
       [0.6472128 ],
       [0.93676674],
       [0.789302  ],
       [0.86216164],
       [0.62404025],
       [0.71118826],
       [0.7806183 ],
       [0.85407937],
       [0.71254855],
       [0.69961745],
       [0.5170797 ],
       [0.640567  ],
       [0.7467193 ],
       [0.7834337 ],
       [0.9613281 ],
       [0.5721727 ],
       [0.7595633 ],
       [0.82380056],
       [0.5080533 ],
       [0.60672885],
       [0.848759  ],
       [0.86495143],
       [0.59955376],
       [0.7454864 ],
       [0.51441467],
       [0.97192806],
       [0.582189  ],
       [0.9243399 ],
       [0.7398719 ],
       [0.5463238 ],
       [0.8363381 ],
       [0.5870365 ],
       [1.0077134 ],
       [0.34056836],
       [0.7872785 ],
       [0.6189204 ],
       [0.57440907],
       [0.64761525],
       [0.89456403],
       [0.31789684],
       [0.68580985],
       [0.7640501 ],
       [1.0316237 ],
       [0.7684349 ],
       [0.31506997],
       [0.57020676],
       [0.573

In [41]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.45683091016658606